## This is the Toronto Neighborhood Assignment within the Coursera Capstone Project

### This assignment is the "Segmenting & Clustering Neighborhoods in Toronto" within the chapter "Applied Data Science" within the IBM Data Science class. Based on the assignment, the goal is to load data for Neighborhoods of Toronto, to integrate the geographical data thereof, select & analyze a neighborhood. Final part of the ssignment is to cluster the neighborhoods. For this last part the instructions allow to use the same methodology as it was used in class for NY.

### Part 1 - load of necessary libraries and data for the analysis

In [61]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('All requested libraries successfully imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

All requested libraries successfully imported.


#### First goal is to download and clean the data so that it can be loaded into a dataframe - data will be loaded from a Wikipedia URL that is referenced in the assignment

In [63]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [64]:
# Converting the URL data
url_data=pd.read_html(url)

In [66]:
# making the data a list, using head to read the first 5 rows
url_data[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [68]:
#Converting the Url data into a dataframe (df)
df = url_data[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [70]:
# To clean up the data we will drop all Postal Code where Borough is Not Assigned
df.replace("Not assigned", np.NaN, inplace = True)
df.dropna(subset=["Borough"], axis=0, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
# Creating a second dataframe for converting the "Neighborhood" to a string format
df_two=df
df_two['Neighborhood'] = df_two['Neighborhood'].astype(str)

##### Re-indexing the list

In [71]:
# Since the first two Boroughs did not have any neighborhoods assigned and got "cleaned up", we need to reset the dataframe
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [80]:
# How many assigned postal codes are there?
df_final = df_two.groupby(['Postal Code','Borough'], as_index=False, sort=False)['Neighborhood'].agg(', '.join)
print(df_final.shape)
df_final.head()

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Result: The data has now be cleaned, loaded into a dataframe and we generated a dataset with three columns and 103 rows.